In [1]:
from preamble import *

In [2]:
file_path = './data/d_pressure_flux_outflow.csv'

In [3]:
data = pd.read_csv(file_path)
data['datetime'] = pd.to_datetime(data['datetime'])
data.head()

,datetime,pressure,flux,outflow
0,2023-01-01 00:01:00,9.69,75.23,101.18
1,2023-01-01 00:02:00,9.69,74.65,98.53
2,2023-01-01 00:03:00,9.69,75.28,102.03
3,2023-01-01 00:04:00,9.69,74.67,100.53
4,2023-01-01 00:05:00,9.69,75.06,104.79


In [4]:
data_one_day = data[:1440]
data_one_day.tail()

,datetime,pressure,flux,outflow
1435,2023-01-01 23:56:00,9.49,145.10,101.84
1436,2023-01-01 23:57:00,9.49,144.48,104.04
1437,2023-01-01 23:58:00,9.49,145.45,101.64
1438,2023-01-01 23:59:00,9.49,144.39,104.83
1439,2023-01-02 00:01:00,9.49,144.38,101.87


In [5]:
# 송수 압력 P (단위: Pa)
# 유량 Q (단위: m³/min)
# 물의 밀도 ρ (약 1000 kg/m³)
# 중력가속도 g (9.81 m/s²)
# 펌프 효율 eta (소수로 표현)

def calculate_power_from_pressure(P, Q_m3_min, eta):
    rho = 1000  # 물의 밀도 (kg/m³)
    g = 9.81    # 중력가속도 (m/s²)

    # 유량을 m³/min에서 m³/s로 변환
    Q_m3_s = Q_m3_min / 60.0

    # 전력 소비량 계산 (킬로와트)
    power_kW = (Q_m3_s * P * 100000) / (eta * 1000)
    print(power_kW)
    return power_kW

P = 2  # 송수 압력 (bar) 가정
Q_m3_min = 32 / 60  # m³/min
eta = 0.85  # 효율

power_kW = calculate_power_from_pressure(P, Q_m3_min, eta)
print(f"전력 소비량: {power_kW} kW")

2.0915032679738563
전력 소비량: 2.0915032679738563 kW


In [6]:
import pickle

with open('best_polynomial_regression_model.pkl', 'rb') as f:
  model, poly = pickle.load(f)

In [7]:
from datetime import datetime, time

# 전기 요금 정보 (인상된 요금 반영)
electricity_rates = {
    'summer': {
        'off_peak': 64.37,  # 경부하 요금 (원/kWh)
        'mid_peak': 92.46,  # 중간부하 요금 (원/kWh)
        'on_peak': 123.88   # 최대부하 요금 (원/kWh)
    },
    'spring_fall': {
        'off_peak': 64.37,  # 경부하 요금 (원/kWh)
        'mid_peak': 69.50,  # 중간부하 요금 (원/kWh)
        'on_peak': 86.88    # 최대부하 요금 (원/kWh)
    },
    'winter': {
        'off_peak': 71.88,  # 경부하 요금 (원/kWh)
        'mid_peak': 90.80,  # 중간부하 요금 (원/kWh)
        'on_peak': 116.47   # 최대부하 요금 (원/kWh)
    }
}

# 시간대 구분
time_periods = {
    'summer': {
        'off_peak': [(time(23, 0), time(9, 0))],
        'mid_peak': [(time(9, 0), time(11, 0)), (time(12, 0), time(13, 0)), (time(17, 0), time(23, 0))],
        'on_peak': [(time(11, 0), time(12, 0)), (time(13, 0), time(17, 0))]
    },
    'spring_fall': {
        'off_peak': [(time(23, 0), time(9, 0))],
        'mid_peak': [(time(9, 0), time(11, 0)), (time(12, 0), time(13, 0)), (time(17, 0), time(23, 0))],
        'on_peak': [(time(11, 0), time(12, 0)), (time(13, 0), time(17, 0))]
    },
    'winter': {
        'off_peak': [(time(23, 0), time(9, 0))],
        'mid_peak': [(time(9, 0), time(10, 0)), (time(12, 0), time(17, 0)), (time(20, 0), time(22, 0))],
        'on_peak': [(time(10, 0), time(12, 0)), (time(17, 0), time(20, 0)), (time(22, 0), time(23, 0))]
    }
}

In [8]:
for _, row in data.iterrows():
  print(row['flux'])
  break

75.22504


In [9]:
def get_season_and_period(dt):
  month = dt.month
  current_time = dt.time()
  
  if month in [7, 8]:
    season = 'summer'
  elif month in [11, 12, 1, 2]:
    season = 'winter'
  else:
    season = 'spring_fall'
  
  for period, times in time_periods[season].items():
    for start, end in times:
      if start <= current_time < end:
        return season, period
  return season, 'off_peak'

def calculate_daily_cost(data, eta = 0.85):
  # data = pd.read_csv(file_path, parse_dates=['datetime'])
  total_cost = 0
  
  for _, row in data.iterrows():
    season, period = get_season_and_period(row['datetime'])
    power_kW = calculate_power_from_pressure(row['pressure'], row['flux'], eta)
    cost_per_kWh = electricity_rates[season][period]
    energy_kWh = power_kW / 60  # converting kW to kWh per minute
    total_cost += energy_kWh * cost_per_kWh
    
  return total_cost

# eta = 0.85
# cost = int(calculate_daily_cost(data_one_day, eta))
# cost_format = format(cost, ',')
# print(f'Total daily electricity cost: {cost_format} KRW')


In [10]:
from sklearn.preprocessing import PolynomialFeatures

def calculate_daily_cost_by_linear(data):
  # data = pd.read_csv(file_path, parse_dates=['datetime'])
  total_cost = 0
  poly = PolynomialFeatures(3)
  for _, row in data.iterrows():
    season, period = get_season_and_period(row['datetime'])
    flux_poly = poly.fit_transform(np.array([[row['flux']]]))
    power_kW = model.predict(flux_poly)[0]
    cost_per_kWh = electricity_rates[season][period]
    energy_kWh = power_kW / 60  # converting kW to kWh per minute
    total_cost += energy_kWh * cost_per_kWh
    print(power_kW)
  return total_cost

In [11]:
cost = calculate_daily_cost_by_linear(data_one_day)
cost_format = format(int(cost), ',')
print(f'Total daily electricity cost: {cost_format} KRW')

122.36671653561484
122.09439552135788
122.39082758617957
122.09941720401811
122.28821772635722
122.06091768443527
122.12225504126134
122.14985583204086
122.1357613357578
122.14699579137371
122.3877968172649
122.0639578547414
122.21716688869466
122.17775649938991
122.03425494535682
122.26367457903862
121.96766860259906
122.08581790120172
122.16978653061625
122.2499303822144
122.13288717340974
122.28433801935323
122.10189945120509
121.86228602339104
122.21206583590742
122.05325303398766
122.20188799823613
122.06858262975062
122.10761351050563
122.14373472637801
122.28232652570468
122.17083230236398
122.24357693383135
122.38194096422907
122.15996895760043
122.14833270012622
121.89220634071363
122.09240979575813
122.03285676533127
122.1921309398448
122.19458090555963
122.23851342664013
122.09102552757642
121.95358222572693
122.45925178125273
122.35961856554293
122.36347581547697
122.89778521460606
133.9946472671973
133.41566615977615
133.66343676119186
133.46409603162556
133.4323661284666


In [12]:
def hourly_simulation(outflow, inflow_rate, v_min, v_max, current_storage):
  test_storage = current_storage
  for minute in range(60):
    test_storage += inflow_rate - outflow[minute]
    if test_storage < v_min:  # 최소 수위를 벗어남
        return 1
    elif test_storage > v_max:  # 최대 수위를 벗어남
        return -1
  return 0  # 기존 유입량 유지 가능

In [13]:
from datetime import timedelta
# 여기서부터 변수 값 보면서 디버깅 해볼 것

# data 내의 칼럼
  # inflow (24시간 동안 분당) 배수지로의 유입량 m^3/min
  # outflow (24시간 동안 분당) 배수지에서 유출량 m^3/min
# v_initial  초기 배수지 저장량(현재 시점 초기값) m^3
# v_min, v_max 배수지 수조의 적정 수위 범위
# 매 정각에 해당 함수를 실행한다는 가정

def optimize_pump_flow(data, v_initial, capacity):
  outflow = data['outflow'].values
  minutes = len(data['datetime'])
  pump_flow = np.zeros(minutes)  # 각 분당 펌프 유량 설정
  over_flow = np.zeros(minutes)  # 넘으면 저장
  lower_flow = np.zeros(minutes)  # 모자라면 저장
  storage = v_initial  # 초기 배수지 저장량
  start_time = data['datetime'][0]
  v_min, v_max = capacity * 0.31, capacity * 0.94 # 1% 보수적 한계
  
  hourly_flow = 0  # 현재 1시간 동안 유지될 유량

  for minute in range(minutes):
    if minute % 60 == 0:  # 정각마다 유량 조정
      start_idx = minute
      end_idx = start_idx + 60
      expected_outflow = np.sum(outflow[start_idx:end_idx]) / 60  # 시간 단위로 변환
      
      # 전기 요금이 싼 심야 시간 우선 활용
      t = minute // 60 
      current_time = start_time + timedelta(hours=t)
      season, period = get_season_and_period(current_time)

      # cost_per_kWh = electricity_rates[season][period]
      
      # 저장량을 유지하기 위한 기본 필요 유입량
      required_inflow = expected_outflow
      
      # 배수지 저장량 확인 및 조정(해당 분의 유입 유출 조정)
      if storage - expected_outflow < v_min:
        hourly_flow = required_inflow + (v_min - (storage - expected_outflow))
      elif storage - expected_outflow > v_max:
        ## 여기서 수위를 넘는데 물을 채운다는건 이후 시뮬레이션에서 무조건 걸린다... 어떡하지 그냥 0으로 받아야 하나나
        hourly_flow = required_inflow - ((storage - expected_outflow) - v_max) 
        # hourly_flow = 0
      else:
        hourly_flow = required_inflow
      
      # print(hourly_flow)
      # 심야 시간 요금 절약을 위해 조정 (추가적인 충전 고려)
      # if period == 'off_peak' and storage + hourly_flow - expected_outflow <= v_max:
      #   print('추가 충전')
      #   hourly_flow += (v_max - (storage + hourly_flow - expected_outflow)) * 0.2 # 추가 충전
      # elif period == 'mid_peak' and storage + hourly_flow - expected_outflow <= v_max:
      #   print('추가 충전')
      #   hourly_flow += (v_max - (storage + hourly_flow - expected_outflow)) * 0.1 # 추가 충전
      if period == 'on_peak' and storage + hourly_flow - expected_outflow >= v_min:
        print('절감 충전')
        hourly_flow -= ((storage + hourly_flow - expected_outflow) - v_min) * 0.2 # 절감 충전

    pump_flow[minute] = max(hourly_flow, 0)
    
    # 배수지에 반영
    storage += pump_flow[minute] - outflow[minute]
    if (storage > v_max):
      over_flow[minute] = storage - v_max
    elif (storage < v_min):
      lower_flow[minute] = v_min - storage

  over_non_zero_indices =  np.nonzero(over_flow)
  lower_non_zero_indices = np.nonzero(lower_flow)

  hourly_over_amount = np.zeros(24)
  hourly_lower_amount = np.zeros(24)
  
  if len(over_non_zero_indices[0]) != 0:
    for idx in over_non_zero_indices[0]:
      print(f'{idx}분 over')
      h = idx // 60
      hourly_over_amount[h] -= over_flow[idx]
  if len(lower_non_zero_indices[0]) != 0:
    for idx in lower_non_zero_indices[0]:
      print(f'{idx}분 lower')
      h = idx // 60
      hourly_lower_amount[h] += lower_flow[idx]

  adjustment = hourly_over_amount + hourly_lower_amount
  # print(adjustment)

  for i, value in enumerate(adjustment):
    if value == 0: continue
    else:
      pump_flow[i * 60 : (i + 1) * 60] += (value / 60)

  return pump_flow

v_initial = 1419
capacity = 2000
optimized_flow = optimize_pump_flow(data_one_day, v_initial, capacity)
print(optimized_flow)
# print(len(optimized_flow))
# 유입량이 음수가 나옴 어디서 음수로 설정되는지 모르겠음 디버깅 해봐야 할 듯
# 상단의 jupyter variables 실행해서 변수 변하는거 한번 보면서 정 안되면 python파일로 추출해서 디버깅 해보자
# hourly_flow가 어딘가에서 음수가 된다는게 관건  
# 시간내에 over된 값들 중에 맥스 값 만을 조정하면 된다

절감 충전
절감 충전
절감 충전
절감 충전
절감 충전
절감 충전
554분 over
555분 over
556분 over
557분 over
558분 over
559분 over
560분 over
561분 over
562분 over
661분 over
662분 over
663분 over
664분 over
665분 over
666분 over
667분 over
668분 over
669분 over
670분 over
671분 over
672분 over
673분 over
674분 over
675분 over
676분 over
677분 over
678분 over
679분 over
680분 over
681분 over
682분 over
683분 over
684분 over
685분 over
686분 over
687분 over
688분 over
689분 over
690분 over
691분 over
692분 over
693분 over
694분 over
695분 over
696분 over
697분 over
698분 over
699분 over
700분 over
701분 over
702분 over
703분 over
704분 over
705분 over
706분 over
707분 over
708분 over
709분 over
710분 over
711분 over
712분 over
713분 over
714분 over
715분 over
716분 over
717분 over
718분 over
719분 over
720분 over
721분 over
722분 over
723분 over
724분 over
725분 over
726분 over
727분 over
728분 over
729분 over
730분 over
731분 over
732분 over
733분 over
734분 over
735분 over
736분 over
737분 over
738분 over
739분 over
740분 over
741분 over
742분 over
743분 over
744분 over
745분 over
746분 over
747분 over
748분

In [14]:
data_opti = data_one_day.copy()
data_opti['flux'] = optimized_flow
data_opti.head()

,datetime,pressure,flux,outflow
0,2023-01-01 00:01:00,9.69,102.79,101.18
1,2023-01-01 00:02:00,9.69,102.79,98.53
2,2023-01-01 00:03:00,9.69,102.79,102.03
3,2023-01-01 00:04:00,9.69,102.79,100.53
4,2023-01-01 00:05:00,9.69,102.79,104.79


In [15]:
cost_opti = calculate_daily_cost_by_linear(data_opti)
cost_format_opti = format(int(cost_opti), ',')

135.869523786468
135.869523786468
135.869523786468
135.869523786468
135.869523786468
135.869523786468
135.869523786468
135.869523786468
135.869523786468
135.869523786468
135.869523786468
135.869523786468
135.869523786468
135.869523786468
135.869523786468
135.869523786468
135.869523786468
135.869523786468
135.869523786468
135.869523786468
135.869523786468
135.869523786468
135.869523786468
135.869523786468
135.869523786468
135.869523786468
135.869523786468
135.869523786468
135.869523786468
135.869523786468
135.869523786468
135.869523786468
135.869523786468
135.869523786468
135.869523786468
135.869523786468
135.869523786468
135.869523786468
135.869523786468
135.869523786468
135.869523786468
135.869523786468
135.869523786468
135.869523786468
135.869523786468
135.869523786468
135.869523786468
135.869523786468
135.869523786468
135.869523786468
135.869523786468
135.869523786468
135.869523786468
135.869523786468
135.869523786468
135.869523786468
135.869523786468
135.869523786468
135.8695237864

In [16]:
saved_cost = cost - cost_opti
saved_cost_format = format(int(saved_cost), ',')
print(f'Total daily electricity cost: {cost_format} KRW')
print(f'Total daily electricity cost used optimization : {cost_format_opti} KRW')
print(f'Save money on electricity bills : {saved_cost_format} KRW')
print(f'electricity bill savings rate : {(1 - (int(cost_opti) / int(cost))) * 100:.2f} % 감소')

Total daily electricity cost: 352,454 KRW
Total daily electricity cost used optimization : 946,905,715,946,249 KRW
Save money on electricity bills : -946,905,715,593,795 KRW
electricity bill savings rate : -268660794201.17 % 감소
